根据所给数据构建样本数据集

In [26]:
import pandas as pd
from functools import reduce
import json

In [27]:
price = pd.read_pickle("stock_price_standard_")
st = pd.read_pickle("stock_st_standard_")
valuation = pd.read_pickle("stock_valuation_standard_")

In [28]:
#price

In [29]:
#price.loc["S.CN.SZSE.300869"]

建立股票代码映射

In [30]:
dt = price.stack()
dt = dt.reset_index()
single_stock = price.groupby(price.index)
code_set = set(list(dt["level_0"]))
num_id = [x for x in range(1, 1411)]
stock_id = dict(zip(num_id, code_set))

In [31]:
# with open("stock-num_id.json", 'w') as file:
#     file.write(json.dumps(stock_id))

获得个股的收盘价

In [32]:
closePrice = single_stock.get_group((stock_id[1], "Close"))
for i in range(2, 1411):
    closePrice = closePrice.append(single_stock.get_group(((stock_id[i], "Close"))))

In [33]:
openPrice = single_stock.get_group((stock_id[1], "Open"))
for i in range(2, 1411):
    openPrice = closePrice.append(single_stock.get_group(((stock_id[i], "Open"))))

In [34]:
#closePrice

## 计算超额收益率

### 1. 计算沪深300收益率作为基准

In [35]:
# 获取沪深300历史数据，数据来自JoinQuant
hs300 = pd.read_csv("hs300.csv")
hs300.index = pd.to_datetime(hs300["Unnamed: 0"])
hs300.drop("Unnamed: 0", axis = 1, inplace = True)
#hs300

获得月初，月末数据,去除2022年2月的数据

In [36]:
hs300_M_end = hs300.loc[hs300.groupby(hs300.index.to_period('M')).apply(lambda x: x.index.max())]
hs300_M_start = hs300.loc[hs300.groupby(hs300.index.to_period('M')).apply(lambda x: x.index.min())]
hs300_M_end = hs300_M_end[:-1]
hs300_M_start = hs300_M_start[:-1]

In [37]:
#hs300_M_start

In [38]:
#hs300_M_end

计算沪深300每月收益率

In [39]:
mon = list(set(hs300.index))
mon.sort()
trade_mon =  [x.strftime("%Y-%m") for x in mon]
func = lambda x,y:x if y in x else x + [y]
trade_mon = reduce(func, [[], ] + trade_mon)
hs300_mon = pd.DataFrame(columns=["trade_mon", "rf"])
hs300_mon["trade_mon"] = trade_mon[:-1]
hs300_mon["rf"] = [(endPrice - initPrice)/initPrice for initPrice, endPrice in zip(hs300_M_start["close"], hs300_M_end["close"])]

In [40]:
#hs300_mon

### 2. 计算每支股票超额收益率，作为label

In [41]:
M_start_date = list(hs300_M_start.index)
M_end_date = list(hs300_M_end.index)

In [42]:
#M_start_date

In [43]:
M_return = pd.DataFrame(columns=["num_id"]+list(hs300_mon["trade_mon"]))

In [44]:
#M_return

In [45]:
M_return["num_id"] = num_id

In [46]:
#M_start_date

In [47]:
#M_closePrice

In [48]:
for i in range(0, 145):
    M_return[list(hs300_mon["trade_mon"])[i]] = list((closePrice.loc[:, M_end_date[i]] - closePrice.loc[:, M_start_date[i]])/closePrice.loc[:, M_start_date[i]])-hs300_mon.loc[i, "rf"]

## 生成样本数据

1月末的ratio等作为2月的feature，以此类推

In [49]:
stock_val = valuation.groupby(valuation.index)

In [50]:
features = list(valuation.index.levels[1])

In [51]:
#features

In [52]:
M_feature = pd.DataFrame(columns=M_end_date)
for num in stock_id:
    for feature in features:
        M_feature = M_feature.append(stock_val.get_group((stock_id[num], feature))[M_end_date])
#M_feature

In [53]:
#features

In [54]:
train_21 = pd.DataFrame(columns=["num_id", "time"] + features + ["rf"])
#train_21

In [55]:
# 日期对应：feature是月末，照应的label是下个月的超额收益
date = list(hs300_mon["trade_mon"])

In [56]:
#len(date)

In [77]:
import math

In [84]:
train_21 = pd.DataFrame(columns=["num_id", "time"] + features + ["rf"])
date = list(hs300_mon["trade_mon"])
tmp1 = []
for i in range(13):
    tmp1 = tmp1 + num_id
    train_21["time"][i*1410: (i+1)*1410] = date[i+132]
train_21["num_id"] = tmp1
if math.isnan(train_21["time"][1]):
    for i in range(13):
        train_21["time"][i*1410: (i+1)*1410] = date[i+132]
train_21

/tmp/ipykernel_5988/3433064262.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_21["time"][i*1410: (i+1)*1410] = date[i+132]


,num_id,time,Capitalization,CirculatingCap,CirculatingMarketCap,MarketCap,PBRatio,PCFRatio,PSRatio,PeRatio,PeRatioLYR,TurnoverRatio,rf
0,1,2021-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2021-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2021-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2021-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2021-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18325,1406,2022-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18326,1407,2022-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18327,1408,2022-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18328,1409,2022-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
#list(M_feature["2010-01-29"][M_feature.index == ("S.CN.SZSE.000525", "Capitalization")])[0]

In [75]:
date_map = dict(zip(date, M_end_date))
date_map2 = dict(zip(date[:-1], date[1:]))

In [76]:
for feature in features:
    for i in range(16920):
        train_21[feature][i] = list(M_feature[date_map[train_21["time"][i]]][M_feature.index == (stock_id[train_21["num_id"][i]], feature)])[0]

for i in range(16920):
    train_21["rf"][i] = M_return[date_map2[train_21["time"][i]]][train_21["num_id"][i]-1]
train_21 = train_21.dropna(axis=0, how='any')

/tmp/ipykernel_5988/2631603481.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_21[feature][i] = list(M_feature[date_map[train_21["time"][i]]][M_feature.index == (stock_id[train_21["num_id"][i]], feature)])[0]


KeyboardInterrupt: 

In [ ]:
#train_21

In [ ]:
#train_21.to_csv("gross_data/2021+.csv")